In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
from collections import defaultdict
import geopandas as gp
from scipy.stats import wilcoxon, pearsonr, spearmanr, kendalltau
from collections import defaultdict
import pandas as pd
from src.constants import BniaIndicators
from src.exp_helper import *
from sklearn.metrics import r2_score, explained_variance_score
from IPython.display import display
from src.utils import cohen_d
pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_columns', 30)

In [3]:
IDCTR2010 = pd.read_excel('data/open-baltimore/raw/VS16_Indicators_2010-2016.xlsx',sheet_name='2010')
IDCTR2010 = IDCTR2010.set_index('CSA2010').drop('Baltimore City')

POP = IDCTR2010['Total Population']

IDCTR = pd.read_excel('data/open-baltimore/raw/VS16_Indicators_2010-2016.xlsx',sheet_name='2015')
IDCTR = IDCTR.set_index('CSA2010').drop('Baltimore City')
print('gender cols is from 2010')
IDCTR = IDCTR.join(IDCTR2010[list(BniaIndicators.gender)])

IDCTR[BniaIndicators.gender[0]] = IDCTR[BniaIndicators.gender[0]]/POP
IDCTR[BniaIndicators.gender[1]] = IDCTR[BniaIndicators.gender[1]]/POP

gender cols is from 2010


In [111]:
COLS = {}
for attr in dir(BniaIndicators):
    if attr.startswith('__'):
        continue
    cols = getattr(BniaIndicators, attr)
    cols_available = [col for col in cols if col in set(IDCTR.columns.tolist())]
    print(attr, len(cols), len(cols_available))
    if len(cols_available)!=0:
        COLS[attr] = cols_available

age 5 5
education 27 13
gender 2 2
household_income 8 8
household_size 4 4
housing 21 20
population 1 0
racial 7 7


In [10]:
TEST_CHOICES = [(pearsonr,'pearsonr'), (spearmanr,'spearmanr'), (kendalltau,'kendalltau')]    

In [11]:
SIG_LVL=0.05

# get pred_rest

In [12]:
def iter_keys():
    for xday in [2, 7]:
        for fpn in ['fp','fn', 'tp']:
            for hname in ['top20', 'above_mean', 'above_mean_std']:
                for normed in [True, False]:
                    yield xday, fpn, hname, normed
def keys2name(xday, fpn, hname, normed):
    s = '%dd-%s-%s' % (xday, fpn, hname)
    if normed:
        return s+'-normed'
    return s + '-raw'

In [13]:
def norm_by_population(df):
    for p in df.index:
#         print(p)
        for c in ['bower','kde200']:
            arr = df.loc[p,c]
            df.loc[p,c]=arr/(POP.values)


In [9]:
# pred_res[xday][fpn][hname][normed]
pred_res = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
for xday, fpn, hname, normed in iter_keys():
    df = pd.read_csv('exp_res/bower_%dday_bnia_%s_hotspots_%s.csv' % (xday, hname, fpn), index_col=0).applymap(eval)
    if normed: 
        norm_by_population(df)
    pred_res[xday][fpn][hname][normed]=df

# stest on stats across all periods

In [17]:
def form_cols(kinds=None):
    cols = []
    if kinds is None:
        kinds = list(COLS.keys())
    if isinstance(kinds, str):
        kinds = [kinds, ]
    for k in kinds:
        v = COLS[k]
        cols.extend([f"{k}**{c}" for c in v])
    return cols

In [74]:
def statistical_test_stat_periods(pred_df, test, idctr_cols):
    stest = []
    # get stats of both methods across periods
    stat_bower = pd.DataFrame(pred_df.bower.tolist()).describe()
    stat_kde200 = pd.DataFrame(pred_df.kde200.tolist()).describe()
#     for stat_name in ['mean','min','max','50%']:
    for stat_name in ['mean',]:
        b_stat = stat_bower.loc[stat_name]
        k_stat = stat_kde200.loc[stat_name]    
        for icol in idctr_cols:
            res = {'stat_name': stat_name, 'indicator': icol}
            # remove the kind of indicators in the cols name
            idctr = IDCTR[icol.split('**')[1]].values
            # bower
            r_bower, p_bower = test(b_stat, idctr)
            res['r_bower'] = r_bower
            res['p_bower'] = p_bower
            res['rp_bower'] = 'r=%0.4f, p=%0.4f' % (r_bower, p_bower)
            # kde 
            r_kde200, p_kde200 = test(k_stat, idctr)
            res['r_kde200'] = r_kde200
            res['p_kde200'] = p_kde200
            res['rp_kde200'] = 'r=%0.4f, p=%0.4f' % (r_kde200, p_kde200)
            # cohen'd
            res['cd_bower'] = cohen_d(b_stat,idctr)
            res['cd_kde200'] = cohen_d(k_stat, idctr)
            stest.append(res)
    return pd.DataFrame(stest)

In [112]:
%%time
idctr_cols= form_cols()
print(len(idctr_cols))
# compute stest result for each combination of xday, fpn, hname, normed
all_stest_stat = {}
for test, tname in TEST_CHOICES:
    stat_idctr_stest_rp = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
    for xday, fpn, hname, normed in iter_keys():
        df = pred_res[xday][fpn][hname][normed]
        stat_idctr_stest_rp[xday][fpn][hname][normed]= statistical_test_stat_periods(df, test, idctr_cols)
    all_stest_stat[tname]=stat_idctr_stest_rp

59
CPU times: user 35.6 s, sys: 56 ms, total: 35.7 s
Wall time: 35.7 s


In [113]:
%%time
kinds = None # 'household_income'
idctr_cols= form_cols(kinds)
name_in_table = 'r'  # rp cd r
sig_lvl = SIG_LVL  # SIG_LVL 1

all_stest_stat_report_table = {}
for _, tname in TEST_CHOICES:
    stat_idctr_stest_rp = all_stest_stat[tname]
    m_stat_sig = []
    for xday, fpn, hname, normed in iter_keys():
        stest = stat_idctr_stest_rp[xday][fpn][hname][normed]
        for mname in ['bower', 'kde200']:
            for stat_name in ['mean',]:
                m_stat_rp = stest[stest.stat_name==stat_name].set_index('indicator').drop('stat_name', axis=1)
                m_stat_rp = m_stat_rp[m_stat_rp[f"p_{mname}"]<sig_lvl][f'{name_in_table}_{mname}']
                m_stat_rp.name = '%s-%s-%s' % (mname, stat_name, keys2name(xday, fpn, hname, normed))
                m_stat_sig.append(m_stat_rp)
            
    report_table =pd.DataFrame(m_stat_sig).T.reindex(idctr_cols).fillna('')
    # handle the display issues with pandas when $ is present
    report_table.index = [col.replace('$','\$') for col in report_table.index]
    # multi level index and columns
    report_table.index = pd.MultiIndex.from_tuples([col.split('**') for col in report_table.index])
    report_table.columns = pd.MultiIndex.from_tuples([col.split('-') for col in report_table.columns])
    all_stest_stat_report_table[tname] = report_table


CPU times: user 656 ms, sys: 0 ns, total: 656 ms
Wall time: 653 ms


In [121]:
tname = 'kendalltau'  # pearsonr spearmanr kendalltau
report_table = all_stest_stat_report_table[tname]
for mname in ('bower','kde200'):
    for stat_name in ['mean',]:
        for xday in (2,7):
            xday_name=f'{xday}d'
            comb_name = '%s-%s-%s-%s' % (tname, mname, stat_name, xday_name)
            print()
            tmp_df = report_table.loc[:, (mname, stat_name, xday_name)].copy()
            tmp_df.index.names = ['kind', 'indicators']
            tmp_df[('count',20,20)] = tmp_df.apply(lambda x: x!='').sum(axis=1)
            tmp_df = tmp_df.reset_index()
            tmp_df.index = [comb_name] * tmp_df.shape[0]
            display(tmp_df)
            print('-'*10)
            print('-'*10)


/home/Jiahui/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


kind                                                                                                indicators        fp                                                                fn                                                                tp                                                         count
                                                                                                                                                         top20           above_mean           above_mean_std               top20           above_mean           above_mean_std               top20           above_mean           above_mean_std              20
                                                                                                                                                        normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    20
kendalltau-bower-mean-2d  age               Percent of Population  Under 5 years old                                                                                                                                                                                                                                                                        0   
kendalltau-bower-mean-2d  age               Percent of Population 5-17 years old                                                                                                                                                                                                                                                                            0   
kendalltau-bower-mean-2d  age               Percent of Population 18-24 years old                                                                               0.19596                                                                                                                                                  0.187879                           2   
kendalltau-bower-mean-2d  age               Percent of Population 25-64 years old                                                                                                                                                                                                                                                                           0   
kendalltau-bower-mean-2d  age               Percent of Population 65 years and over                                                                  -0.228283           -0.197306            -0.245791      -0.206734                                 0.200741                          -0.232323 -0.194613 -0.235017            -0.264646      -0.220202  10  
kendalltau-bower-mean-2d  education         Number of Students Ever Attended 1st - 5th Grade                                                                    0.369148             0.402829                 0.292354            0.286965                                      0.37319             0.288313             0.320647                 0.270798  8   
kendalltau-bower-mean-2d  education         Number of Students Ever Attended 6th - 8th Grade                                                                    0.352981             0.405524                 0.246548            0.316605             0.189353                 0.386662            0.250589             0.293702                 0.230381  9   
kendalltau-bower-mean-2d  education         Number of Students Ever Attended 9th - 12th Grade                                                                   0.365352             0.401753                 0.250758            0.318167             0.198921                 0.401753            0.254803             0.306033                 0.221099  9   
kendalltau-bower-mean-2d  education         Percent of Students that are African American                                                                  

----------
----------



kind                                                                                                indicators        fp                                                                fn                                                                tp                                                         count
                                                                                                                                                         top20           above_mean           above_mean_std               top20           above_mean           above_mean_std               top20           above_mean           above_mean_std              20
                                                                                                                                                        normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    20
kendalltau-bower-mean-7d  age               Percent of Population  Under 5 years old                                                                                                                                                                                                                                                                        0   
kendalltau-bower-mean-7d  age               Percent of Population 5-17 years old                                                                                                                                                                                                                                                                            0   
kendalltau-bower-mean-7d  age               Percent of Population 18-24 years old                                                                               0.197306                                                                                                                                                 0.183838                           2   
kendalltau-bower-mean-7d  age               Percent of Population 25-64 years old                                                                                                                                                                                                                                                                           0   
kendalltau-bower-mean-7d  age               Percent of Population 65 years and over                                                                  -0.232323           -0.19596             -0.248485      -0.208081                                 0.212121                          -0.239057 -0.194613 -0.23367   -0.193266 -0.274074      -0.218855  11  
kendalltau-bower-mean-7d  education         Number of Students Ever Attended 1st - 5th Grade                                                                    0.370495             0.401482                 0.296396            0.264062                                      0.388009            0.307174             0.324688                 0.27484   8   
kendalltau-bower-mean-7d  education         Number of Students Ever Attended 6th - 8th Grade                                                                    0.351634             0.404177                 0.253284            0.299091             0.196699                 0.401482            0.269451             0.295049                 0.231728  9   
kendalltau-bower-mean-7d  education         Number of Students Ever Attended 9th - 12th Grade                                                                   0.366701             0.400405                 0.260196            0.297944             0.206269                 0.408494            0.268285             0.304685                 0.225143  9   
kendalltau-bower-mean-7d  education         Percent of Students that are African American                                                                  

----------
----------



kind                                                                                                indicators        fp                                                                fn                                                                tp                                                         count
                                                                                                                                                          top20           above_mean           above_mean_std               top20           above_mean           above_mean_std               top20           above_mean           above_mean_std              20
                                                                                                                                                         normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    20
kendalltau-kde200-mean-2d  age               Percent of Population  Under 5 years old                                                                                                                                                                                                                                                                        0   
kendalltau-kde200-mean-2d  age               Percent of Population 5-17 years old                                                                                                                                                                                                                                                                            0   
kendalltau-kde200-mean-2d  age               Percent of Population 18-24 years old                                                                               0.213468                                      0.193266                                                                                                   0.185185                           3   
kendalltau-kde200-mean-2d  age               Percent of Population 25-64 years old                                                                                                                                                                                                                                                                           0   
kendalltau-kde200-mean-2d  age               Percent of Population 65 years and over                                                                  -0.23771            -0.197306            -0.265993      -0.22963                       0.183838   0.240404                          -0.230976 -0.201347 -0.239057  -0.194613 -0.241321      -0.200876  12  
kendalltau-kde200-mean-2d  education         Number of Students Ever Attended 1st - 5th Grade                                                                    0.375884             0.413607                 0.282924            0.213612                                      0.338161            0.295049             0.334119                 0.251517  8   
kendalltau-kde200-mean-2d  education         Number of Students Ever Attended 6th - 8th Grade                                                                    0.338161             0.418996                 0.239811            0.245957                                      0.354328            0.26002              0.30448                  0.20027   8   
kendalltau-kde200-mean-2d  education         Number of Students Ever Attended 9th - 12th Grade                                                                   0.355915             0.412538                 0.230536            0.25826                                       0.372093            0.253455             0.311426                 0.18691   8   
kendalltau-kde200-mean-2d  education         Percent of Students that are African American                                                       

----------
----------



kind                                                                                                indicators        fp                                                                fn                                                                tp                                                         count
                                                                                                                                                          top20           above_mean           above_mean_std               top20           above_mean           above_mean_std               top20           above_mean           above_mean_std              20
                                                                                                                                                         normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    normed       raw     normed       raw         normed       raw    20
kendalltau-kde200-mean-7d  age               Percent of Population  Under 5 years old                                                                                                                                                                                                                                                                        0   
kendalltau-kde200-mean-7d  age               Percent of Population 5-17 years old                                                                                                                                                                                                                                                                            0   
kendalltau-kde200-mean-7d  age               Percent of Population 18-24 years old                                                                               0.213468                                      0.193266                                                                              0.183838                                                3   
kendalltau-kde200-mean-7d  age               Percent of Population 25-64 years old                                                                                                                                                                                                                                                                           0   
kendalltau-kde200-mean-7d  age               Percent of Population 65 years and over                                                                  -0.241751 -0.185185 -0.198653            -0.268687      -0.22963                                  0.243098                          -0.226936 -0.206734 -0.240404  -0.185185 -0.237277      -0.203573  12  
kendalltau-kde200-mean-7d  education         Number of Students Ever Attended 1st - 5th Grade                                                                    0.378579             0.409566                 0.288313            0.208825                                      0.336814            0.295049             0.335467                 0.240728  8   
kendalltau-kde200-mean-7d  education         Number of Students Ever Attended 6th - 8th Grade                                                                    0.340856             0.414955                 0.2452              0.243853                                      0.355675            0.26002              0.300438                 0.194875  8   
kendalltau-kde200-mean-7d  education         Number of Students Ever Attended 9th - 12th Grade                                                                   0.358612             0.408494                 0.235929            0.256151                                      0.365352            0.253455             0.312774                 0.184211  8   
kendalltau-kde200-mean-7d  education         Percent of Students that are African American                                                       

----------
----------


# test per period

In [59]:
def statistical_test_per_period(pred_df, test, idctr_cols):
    stest = []
    for period, (bower, kde200) in pred_df.iterrows():
        for icol in idctr_cols:
            res = {'period': period, 'indicator': icol}
            idctr = IDCTR[icol].values
            # bower
            r_bower, p_bower = test(bower, idctr)
            res['r_bower'] = r_bower
            res['p_bower'] = p_bower
            res['rp_bower'] = 'r=%0.4f, p=%0.4f' % (r_bower, p_bower)
            # kde 
            r_kde200, p_kde200 = test(kde200, idctr)
            res['r_kde200'] = r_kde200
            res['p_kde200'] = p_kde200
            res['rp_kde200'] = 'r=%0.4f, p=%0.4f' % (r_kde200, p_kde200)
            stest.append(res)
    
    return pd.DataFrame(stest)

In [60]:
def keep_sig_per_period(df, sig_lvl, mname, periods):
    pivoted = df[df['p_'+mname]<sig_lvl].pivot(index='period', columns='indicator', values='rp_'+mname) 
    return pivoted.reindex(periods)
def sig_pcnt_periods(df, sig_lvl, mname, periods):
    ires = keep_sig_per_period(df, sig_lvl, mname, periods)
    return (~ires.isnull()).mean()

In [61]:
all_periods = {2: pred_res[2]['fn']['above_mean_std'][False].index, 7: pred_res[7]['fn']['above_mean_std'][False].index} # key: xday

In [65]:
%%time
sig_lvl = 0.05
cols = COLS['household_income']
sum_table_each_test = {}
for test, tname in TEST_CHOICES:
    period_idctr_stest_rp = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
    for xday, fpn, hname, normed in iter_keys():
        df = pred_res[xday][fpn][hname][normed]
        period_idctr_stest_rp[xday][fpn][hname][normed]= statistical_test_per_period(df, test, cols)
    sig_pcnt_of_each_method = []
    for mname in ['bower', 'kde200']:
        for xday, fpn, hname, normed in iter_keys():
            stest = period_idctr_stest_rp[xday][fpn][hname][normed]
            sig_pcnt = sig_pcnt_periods(stest, sig_lvl, mname, all_periods[xday])
            sig_pcnt.name = '%s-%s' % (mname, keys2name(xday, fpn, hname, normed))
            sig_pcnt_of_each_method.append(sig_pcnt)
    sum_table =pd.DataFrame(sig_pcnt_of_each_method).T.reindex(cols).fillna(0)
    sum_table.index = [col.replace('$','\$') for col in sum_table.index]
    tuples = [col.split('-') for col in sum_table.columns]
    index = pd.MultiIndex.from_tuples(tuples)
    sum_table.columns = index
    sum_table_each_test[tname] = sum_table

CPU times: user 3min 16s, sys: 20 ms, total: 3min 16s
Wall time: 3min 16s


In [69]:
tname = 'kendalltau'  # pearsonr spearmanr kendalltau
sum_table = sum_table_each_test[tname]
for mname in ('bower','kde200'):
    for xday in ('2d','7d'):
        print()
        tmp_df = sum_table.loc[:, (mname, xday)].copy()
        tmp_df.index.name = '%s-%s-%s' % (tname, mname, xday)
        display(tmp_df)
        print('    ')
        print('---------------------')

fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.069536  0.049669   
Percent of Households Earning Less than \$25,000    0.195364  0.079470   
Percent of Households Earning \$25,000 to \$40,000  0.009934  0.052980   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.109272  0.006623   
Percent of Households Earning More than \$75,000    0.145695  0.178808   
Percent of Family Households Living Below the P...  0.516556  0.417219   
Percent of Children Living Below the Poverty Line   0.582781  0.460265   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.145695  0.069536   
Percent of Households Earning Less than \$25,000     0.264901  0.039735   
Percent of Households Earning \$25,000 to \$40,000   0.003311  0.072848   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006623   
Percent of Households Earning \$60,000 to \$75,000   0.069536  0.000000   
Percent of Households Earning More than \$75,000     0.331126  0.357616   
Percent of Family Households Living Below the P...   0.552980  0.417219   
Percent of Children Living Below the Poverty Line    0.658940  0.486755   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-2d                                                           
Median Household Income                                  0.026490  0.019868   
Percent of Households Earning Less than \$25,000         0.079470  0.039735   
Percent of Households Earning \$25,000 to \$40,000       0.029801  0.019868   
Percent of Households Earning \$40,000 to \$60,000       0.006623  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.036424  0.009934   
Percent of Households Earning More than \$75,000         0.023179  0.033113   
Percent of Family Households Living Below the P...       0.314570  0.301325   
Percent of Children Living Below the Poverty Line        0.354305  0.258278   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-2d                                                      
Median Household Income                             0.046358  0.046358   
Percent of Households Earning Less than \$25,000    0.056291  0.059603   
Percent of Households Earning \$25,000 to \$40,000  0.049669  0.043046   
Percent of Households Earning \$40,000 to \$60,000  0.046358  0.049669   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.026490   
Percent of Households Earning More than \$75,000    0.046358  0.029801   
Percent of Family Households Living Below the P...  0.076159  0.062914   
Percent of Children Living Below the Poverty Line   0.082781  0.072848   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-2d                                                       
Median Household Income                              0.049669  0.039735   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.074074  0.067340   
Percent of Households Earning Less than \$25,000    0.198653  0.090909   
Percent of Households Earning \$25,000 to \$40,000  0.010101  0.057239   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.117845  0.006734   
Percent of Households Earning More than \$75,000    0.154882  0.191919   
Percent of Family Households Living Below the P...  0.518519  0.434343   
Percent of Children Living Below the Poverty Line   0.585859  0.464646   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.121212  0.084175   
Percent of Households Earning Less than \$25,000     0.252525  0.040404   
Percent of Households Earning \$25,000 to \$40,000   0.003367  0.074074   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.006734   
Percent of Households Earning \$60,000 to \$75,000   0.077441  0.000000   
Percent of Households Earning More than \$75,000     0.319865  0.350168   
Percent of Family Households Living Below the P...   0.542088  0.417508   
Percent of Children Living Below the Poverty Line    0.653199  0.491582   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-bower-7d                                                           
Median Household Income                                  0.023569  0.020202   
Percent of Households Earning Less than \$25,000         0.077441  0.043771   
Percent of Households Earning \$25,000 to \$40,000       0.030303  0.020202   
Percent of Households Earning \$40,000 to \$60,000       0.006734  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.037037  0.010101   
Percent of Households Earning More than \$75,000         0.023569  0.033670   
Percent of Family Households Living Below the P...       0.319865  0.306397   
Percent of Children Living Below the Poverty Line        0.373737  0.262626   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-bower-7d                                                      
Median Household Income                             0.043771  0.030303   
Percent of Households Earning Less than \$25,000    0.033670  0.023569   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.090909   
Percent of Households Earning \$40,000 to \$60,000  0.026936  0.040404   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.006734   
Percent of Households Earning More than \$75,000    0.053872  0.050505   
Percent of Family Households Living Below the P...  0.043771  0.030303   
Percent of Children Living Below the Poverty Line   0.050505  0.033670   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-bower-7d                                                       
Median Household Income                              0.057239  0.047138   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.036424  0.049669   
Percent of Households Earning Less than \$25,000    0.149007  0.109272   
Percent of Households Earning \$25,000 to \$40,000  0.000000  0.046358   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.251656  0.000000   
Percent of Households Earning More than \$75,000    0.079470  0.231788   
Percent of Family Households Living Below the P...  0.579470  0.490066   
Percent of Children Living Below the Poverty Line   0.695364  0.543046   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.142384  0.149007   
Percent of Households Earning Less than \$25,000     0.254967  0.086093   
Percent of Households Earning \$25,000 to \$40,000   0.000000  0.052980   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.016556   
Percent of Households Earning \$60,000 to \$75,000   0.052980  0.000000   
Percent of Households Earning More than \$75,000     0.307947  0.324503   
Percent of Family Households Living Below the P...   0.609272  0.509934   
Percent of Children Living Below the Poverty Line    0.758278  0.526490   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-2d                                                          
Median Household Income                                  0.019868  0.009934   
Percent of Households Earning Less than \$25,000         0.165563  0.119205   
Percent of Households Earning \$25,000 to \$40,000       0.000000  0.013245   
Percent of Households Earning \$40,000 to \$60,000       0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.105960  0.003311   
Percent of Households Earning More than \$75,000         0.056291  0.066225   
Percent of Family Households Living Below the P...       0.367550  0.370861   
Percent of Children Living Below the Poverty Line        0.410596  0.374172   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-2d                                                     
Median Household Income                             0.036424  0.033113   
Percent of Households Earning Less than \$25,000    0.036424  0.033113   
Percent of Households Earning \$25,000 to \$40,000  0.043046  0.066225   
Percent of Households Earning \$40,000 to \$60,000  0.052980  0.059603   
Percent of Households Earning \$60,000 to \$75,000  0.029801  0.033113   
Percent of Households Earning More than \$75,000    0.043046  0.036424   
Percent of Family Households Living Below the P...  0.076159  0.059603   
Percent of Children Living Below the Poverty Line   0.066225  0.046358   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-2d                                                      
Median Household Income                              0.026490  0.026490   
Percent of Households Earning Less than \$25,000     0.

    
---------------------



fp            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.037037  0.047138   
Percent of Households Earning Less than \$25,000    0.154882  0.114478   
Percent of Households Earning \$25,000 to \$40,000  0.000000  0.047138   
Percent of Households Earning \$40,000 to \$60,000  0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000  0.235690  0.000000   
Percent of Households Earning More than \$75,000    0.080808  0.249158   
Percent of Family Households Living Below the P...  0.589226  0.501684   
Percent of Children Living Below the Poverty Line   0.713805  0.552189   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.148148  0.151515   
Percent of Households Earning Less than \$25,000     0.252525  0.094276   
Percent of Households Earning \$25,000 to \$40,000   0.000000  0.053872   
Percent of Households Earning \$40,000 to \$60,000   0.000000  0.013468   
Percent of Households Earning \$60,000 to \$75,000   0.053872  0.000000   
Percent of Households Earning More than \$75,000     0.303030  0.313131   
Percent of Family Households Living Below the P...   0.599327  0.501684   
Percent of Children Living Below the Poverty Line    0.754209  0.531987   

                                                                             \
                                                   above_mean_std             
                                                           normed       raw   
kendalltau-kde200-7d                                                          
Median Household Income                                  0.023569  0.006734   
Percent of Households Earning Less than \$25,000         0.164983  0.117845   
Percent of Households Earning \$25,000 to \$40,000       0.000000  0.013468   
Percent of Households Earning \$40,000 to \$60,000       0.000000  0.000000   
Percent of Households Earning \$60,000 to \$75,000       0.107744  0.000000   
Percent of Households Earning More than \$75,000         0.053872  0.067340   
Percent of Family Households Living Below the P...       0.373737  0.377104   
Percent of Children Living Below the Poverty Line        0.420875  0.380471   

                                                          fn            \
                                                       top20             
                                                      normed       raw   
kendalltau-kde200-7d                                                     
Median Household Income                             0.040404  0.023569   
Percent of Households Earning Less than \$25,000    0.033670  0.016835   
Percent of Households Earning \$25,000 to \$40,000  0.047138  0.067340   
Percent of Households Earning \$40,000 to \$60,000  0.033670  0.060606   
Percent of Households Earning \$60,000 to \$75,000  0.006734  0.010101   
Percent of Households Earning More than \$75,000    0.043771  0.040404   
Percent of Family Households Living Below the P...  0.040404  0.016835   
Percent of Children Living Below the Poverty Line   0.047138  0.043771   

                                                                         \
                                                   above_mean             
                                                       normed       raw   
kendalltau-kde200-7d                                                      
Median Household Income                              0.033670  0.020202   
Percent of Households Earning Less than \$25,000     0.

    
---------------------
